In [1]:
import logging
import numpy as np

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s. %(levelname)s in %(funcName)s: %(message)s')

In [2]:
pwd()

u'D:\\FF\\code\\src'

In [3]:
ls ..\generate

 Volume in drive D is Data
 Volume Serial Number is 1406-ADC1

 Directory of D:\FF\code\generate

13/10/2016  12:58 PM    <DIR>          .
13/10/2016  12:58 PM    <DIR>          ..
13/10/2016  11:14 AM           134,525 bay.mat
13/10/2016  11:13 AM               940 fvm.m
14/10/2016  06:14 PM             1,090 getA.m
13/10/2016  11:14 AM           128,621 mass.txt
13/10/2016  11:13 AM           205,646 stiff.txt
13/10/2016  11:13 AM             1,459 stokes.edp
13/10/2016  11:14 AM            18,225 u.txt
13/10/2016  11:14 AM            19,151 v.txt
13/10/2016  11:14 AM               836 write_csr_matrix.m
               9 File(s)        510,493 bytes
               2 Dir(s)  338,544,721,920 bytes free


In [48]:
def readVectFF(f):
    """Reads a vector in the form output by FreeFem++.
    
    First line contains the number of entries: nbcoef. 
    Next lines contain the entries, with 5 entries per line, 
    separated by tab.
    Entries are read left to right, then line by line.
    """
    
    nbcoef = int(f.next().strip())
    logging.info("attempt to read %s values", nbcoef)
    arr = np.loadtxt(f)
    arr = arr.flatten(order='C')
    if len(arr) != nbcoef:
        errMsg = "readVectFF: Inconsistent dimension, expected %d, got %d" %(nbcoef, len(arr))
        logging.error(errMsg)
        raise RuntimeError(errMsg)
    logging.info("successfully read %s values", nbcoef)
    return arr
        

In [49]:
with open('..\generate\u.txt', 'r') as f:
    x = readVectFF(f)

In [50]:
x

array([ 0.02493221,  0.00731244,  0.05315321, ...,  0.41820378,
        0.42036873,  0.41557122])

In [51]:
len(x)

1370

In [55]:
def readSparseFF(f):
    """Reads a sparse matrix in the form (Morse) output by FreeFem++.
    
    First line contains: n m (is symmetic) nbcoef 
    Next lines contain, for each nonzero coefficient:   i j a_ij where (i,j) \in  {1,...,n}x{1,...,m} 
    """
    
    # strip comment header
    row = f.next().strip()
    while row[0] == '#':
        row = f.next().strip()
    # read shape
    n, m, issym, nbcoef = map(int, row.split())
    logging.info("sparse (%s x %s), sym? %s, attempt to read %s values", n, m, issym, nbcoef)
    # read nzeros
    ijv = np.loadtxt(f)
    nrows, ncols = ijv.shape
    # check dims
    if nrows != nbcoef:
        errMsg = "Inconsistent dimensions, expected %d, got %d" %(nbcoef, nrows)
        logging.error(errMsg)
        raise RuntimeError(errMsg)
    if ncols != 3:
        errMsg = "Inconsistent dimensions, expected %d, got %d" %(3, ncols)
        logging.error(errMsg)
        raise RuntimeError(errMsg)
    # populate matrix
    res = np.zeros((n,m))
    for ifloat,jfloat,v in ijv:
        i, j = int(ifloat) - 1, int(jfloat) - 1  # python is 0-based
        res[i,j] = v
        if issym != 0:
            res[j,i] = v

    logging.info("successfully read %s values", nbcoef)
    return res
        

In [56]:
with open('..\generate\stiff.txt', 'r') as f:
    s = readSparseFF(f)

In [57]:
s.shape

(736L, 736L)